# Análise de reclamações

Autor: Gabriel Bhering

In [1]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

Os dados são provenientes do [kaggle](https://www.kaggle.com/gerosa/procon), e um [script](https://gist.github.com/bhering/d21097f5b125d65fd8c422d81695d7f7) foi rodado para higieniza-los. O que o script faz é remover algumas incoerencias (como traços no campo de CEP), e apagar campos sem dados que estavam preenchidos com `NULL` ou `Nao se aplica`. Com isso, a biblioteca consegue lidar melhor com os dados brutos. Abaixo estão algumas configurações para otimizar a leitura dos arquivos.

In [2]:
dtypes={
    'AnoCalendario': 'float',
    'DataArquivamento': 'str',
    'DataAbertura': 'str',
    'CodigoRegiao': 'float',
    'Regiao': 'str',
    'UF': 'str',
    'strRazaoSocial': 'str',
    'strNomeFantasia': 'str',
    'Tipo': 'float',
    'NumeroCNPJ': 'float',
    'RadicalCNPJ': 'float',
    'RazaoSocialRFB': 'str',
    'NomeFantasiaRFB': 'str',
    'CNAEPrincipal': 'float',
    'DescCNAEPrincipal': 'str',
    'Atendida': 'str',
    'CodigoAssunto': 'float',
    'DescricaoAssunto': 'str',
    'CodigoProblema': 'float',
    'DescricaoProblema': 'str',
    'SexoConsumidor': 'str',
    'FaixaEtariaConsumidor': 'str',
    'CEPConsumidor': 'float',
}
parse_dates=['DataArquivamento', 'DataAbertura']

Todos os .csv da pasta são lidos e compilados em um único dataframe `df`...

In [6]:
dataset_folder='./datasets/costumer_complaints'
all_files=glob.glob(os.path.join(dataset_folder,"*.csv"))
df = pd.concat((pd.read_csv(f, dtype=dtypes, parse_dates=parse_dates) for f in all_files))